Here we demonstrate using actual mixins that have already been defined
to calculate physically real values.

Many mixins exist throughout the stack.  They can be found in the directory

    `sims_catUtils/python/lsst/sims/catUtils/mixins/

There you will find the files

* AstrometryMixin.py  -- mixins to calculate and transform objects' positions on both the sky and the camera

* PhotometryMixin.py -- mixins to calculate the magnitudes of point and extended sources

* EBVmixin.py -- mixin that interfaces with the SFD dust maps

* VariabilityMixin.py -- mixin encoding photometric variability of sources

* CosmologyMixin.py -- mixin to calculate the cosmologial distance modulus

Below, we demonstrate an <b>InstanceCatalog</b> class that uses the photometry
and astrometry mixins to calculate physically meaningful column values

In [ ]:
import numpy
from lsst.sims.catalogs.generation.db import CatalogDBObject
from lsst.sims.utils import ObservationMetaData
from lsst.sims.utils import haversine
from lsst.sims.catUtils.baseCatalogModels import *
from lsst.sims.catalogs.measures.instance import InstanceCatalog, cached
from lsst.sims.catUtils.mixins import AstrometryStars, PhotometryStars

class TutorialCatalog(InstanceCatalog, AstrometryStars, PhotometryStars):

    column_outputs = ['raJ2000', 'decJ2000', 'lsst_u', 'raObserved', 'decObserved',
                      'shift']
    #to see where lsst_u comes from, see the PhotometryStars class in
    #sims_photUtils/python/lsst/sims/photUtils/Photometry.py
    #
    #to see where raObserved and decObserved come from, see the AstrometryStars class in
    #sims_coordUtils/python/lsst/sims/coordUtils/Astrometry.py


    #transform all of the angles into degrees
    transformations = {'raJ2000':numpy.degrees, 'decJ2000':numpy.degrees,
                      'raObserved':numpy.degrees, 'decObserved':numpy.degrees,
                       'shift':numpy.degrees}


    #a handle to be passed to CatalogDBObject.getCatalog() (see tutorial02)
    catalog_type = 'tutorial_catalog'


    @cached
    def get_shift(self):
        """
        A getter for the angular distance between the unrefracted raJ2000, decJ2000
        and the corrected raObserved, decObserved

        Note that because all angles are handled inside of the stack as radians,
        the returned angular distance will also be in radians (though the InstanceCatalog
        daughter class converts it to degrees with the self.transformations dict)
        """
        r0 = self.column_by_name('raJ2000')
        d0 = self.column_by_name('decJ2000')
        r1 = self.column_by_name('raObserved')
        d1 = self.column_by_name('decObserved')

        return haversine(r0, d0, r1, d1)

Now write the catalog (remember: <b>do not run the cell above more than once without restarting the kernel; it will throw an ugly-looking error</b>)

In [ ]:
myDB = CatalogDBObject.from_objid('allstars')
obs_metadata = ObservationMetaData(unrefractedRA=220.0, unrefractedDec=19.0,
                                   boundType='circle', boundLength=0.1,
                                   mjd=52000.0)

cat = TutorialCatalog(myDB, obs_metadata=obs_metadata)
cat.write_catalog('tutorial_astrometry_photometry.txt')

And, just for kicks, use `CatalogDBObject.getCatalog()`.

In [ ]:
obs_metadata = ObservationMetaData(unrefractedRA=120.0, unrefractedDec=-5.0,
                                   boundType='circle', boundLength=0.1,
                                   mjd=52000.0)

cat = myDB.getCatalog('tutorial_catalog', obs_metadata=obs_metadata)
cat.write_catalog('tutorial_astrometry_photometry_get.txt')